In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt2
import pandas as pd
from pandas import datetime
import math, time
import itertools
from sklearn import preprocessing
import datetime
import quandl
import tensorflow as tf

In [2]:
# Load the data
top10_data = pd.read_csv("return_data_top10.csv")

# Equalize the data
min_date = pd.to_datetime('2013-10-1')
top10_data = top10_data[top10_data.Date >= '2013-10-1']

In [3]:
df = pd.pivot_table(data=top10_data, columns=["Date"], index=["Ticker"], values="forward_return")
df

Date,2013-10-10,2013-10-11,2013-10-14,2013-10-15,2013-10-16,2013-10-17,2013-10-18,2013-10-21,2013-10-22,2013-10-23,...,2017-09-15,2017-09-18,2017-09-19,2017-09-20,2017-09-21,2017-09-22,2017-09-25,2017-09-26,2017-09-27,2017-09-28
Ticker,,,,,,,,,,,,,,,,,,,,,
AAPL,0.006482,0.006550,0.005322,0.004881,0.006757,0.008702,0.024508,-0.002867,0.009796,0.013239,...,-0.007568,0.000378,-0.016758,-0.017172,-0.009779,-0.008822,0.017204,0.007118,-0.006160,0.005480
BRK_B,0.008447,0.001641,-0.011810,0.017971,0.001971,0.000428,-0.000684,0.005133,-0.009192,0.003694,...,0.008783,0.008486,0.004426,-0.003590,-0.007098,0.003629,-0.003616,0.007753,-0.000327,0.000600
FB,0.001223,0.008145,-0.000202,0.033030,0.021023,0.038498,-0.006824,-0.021820,-0.014713,0.010501,...,-0.009150,0.014764,-0.002029,-0.006157,-0.003331,-0.044975,0.008227,0.021131,0.006262,0.012327
GE,0.006186,-0.000820,-0.007793,0.007028,0.013136,0.035251,0.023092,-0.004591,-0.012298,0.009339,...,0.022148,-0.010630,0.004959,0.017681,0.004848,0.009650,-0.007168,-0.022463,-0.005334,-0.002475
JNJ,0.019025,0.003913,0.001448,0.013121,0.009439,-0.003697,-0.004693,0.012719,-0.002815,0.002714,...,0.006917,-0.001182,-0.014791,-0.011034,-0.002732,-0.001674,-0.001753,-0.009088,-0.002158,0.004171
MSFT,0.010960,0.009376,0.001161,0.004349,0.008083,0.001145,0.000858,-0.011718,-0.023713,-0.001185,...,-0.001992,0.003725,-0.006628,-0.009741,0.002695,-0.015455,0.000000,0.008054,0.000271,0.008393
PG,0.007575,0.003313,-0.014478,0.017287,0.013786,-0.000126,-0.005541,0.017855,0.006594,-0.003708,...,-0.001287,0.010950,0.002442,-0.018644,-0.004318,0.005204,-0.000755,-0.019212,0.000220,0.000990
T,0.001464,-0.008187,-0.006191,0.014536,0.007018,0.004936,0.017625,0.000284,0.001391,-0.018396,...,0.008625,0.021112,0.005758,-0.005464,0.009681,0.013734,-0.010225,0.001291,0.006964,0.003330
WFC,-0.000241,0.007724,-0.005030,0.016129,0.011135,0.000000,-0.001406,0.007508,-0.004192,-0.001637,...,0.020325,0.012332,0.007309,0.005767,0.003515,-0.004055,-0.004257,0.007621,0.000738,0.016590


# LSTM + Dropout + LSTM 

In [4]:
#https://github.com/tensorflow/nmt#training--how-to-build-our-first-nmt-system

# Parameters
learning_rate = 0.0001
training_epochs = 10000
display_step = 100

# Network Parameters
hidden_dim = 4
batch_size = 1
source_sequence_length = 500
num_tickers = 10
_time_major = True 
Pin = 1.0
Pout = 0.80
Pstate = 0.80
"""
 If true, these Tensors must be shaped [max_time, batch_size, depth]. 
 If false, these Tensors must be shaped [batch_size, max_time, depth]. 
 Using time_major = True is a bit more efficient
"""

# Reset the graph
tf.reset_default_graph()

# Graph Placeholders
encoder_inputs = tf.placeholder(shape=(source_sequence_length, None, num_tickers), 
                                dtype=tf.float64, name='encoder_inputs')
decoder_targets = tf.placeholder(shape=(source_sequence_length, None, num_tickers), 
                                 dtype=tf.float64, name='decoder_targets')

# Build Encoder cell
encoder_cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_dim,
                                       forget_bias=1.0,
                                       state_is_tuple=True,
                                       activation=tf.tanh,
                                       reuse=None)

# Introduce Dropout to the Encoding
encoder_dropout = tf.contrib.rnn.DropoutWrapper(encoder_cell,
                                                input_keep_prob=Pin,
                                                output_keep_prob=Pout,
                                                state_keep_prob=Pstate)

# Build Simple Decoder Cell
decoder_cell = tf.nn.rnn_cell.LSTMCell(num_units=num_tickers,
                                       forget_bias=1.0,
                                       state_is_tuple=True,
                                       activation=tf.tanh,
                                       reuse=None)

# Compile Deep Network
autoencoder_layers = tf.nn.rnn_cell.MultiRNNCell([encoder_dropout, decoder_cell], 
                                             state_is_tuple=True)


# Run Dynamic RNN
#   encoder_outpus: [batch_size, max_time, num_units]
#   encoder_state: [batch_size, num_units]
outputs, state = tf.nn.dynamic_rnn(autoencoder_layers,
                                   encoder_inputs,
                                   sequence_length=[source_sequence_length],
                                   time_major=_time_major,
                                   dtype=tf.float64)

# Loss Metric (unregularized)
loss = tf.losses.mean_squared_error(labels=decoder_targets, predictions=outputs)
loss = tf.reduce_mean(loss)

# Optimizer
trainer = tf.train.AdamOptimizer(learning_rate=learning_rate)
gradients = trainer.compute_gradients(loss)
gradients_clipped = [(tf.clip_by_value(t[0],-1,1),t[1]) for t in gradients]
optimizer = trainer.apply_gradients(gradients_clipped)


In [10]:
# Training function

"""
The loss does not change significantly, so I reduce the # of step into 500.
"""
num_steps = 500
x_train = df.values[0:, :500].T.reshape((500, 1, 10))
y_train = df.values[0:, :500].T.reshape((500, 1, 10))

tic = time.time()        # Initialize time count
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(num_steps):
        _, train_loss, pred = sess.run([optimizer, loss, outputs], feed_dict={encoder_inputs:x_train, 
                                                               decoder_targets:y_train})

        if (step % 50 == 0):
            val_loss = sess.run([loss], feed_dict={encoder_inputs:x_train, 
                                                     decoder_targets:y_train})
            
            print("-"*50)
            print("Training step: {0} - Loss = {1}".format(step, val_loss))
        if (step % 100 == 0):
#             pred = sess.run([outputs], feed_dict={encoder_inputs:x_train})
            print("predition: ", pred)

    print("="*50)
    
toc = time.time()
print("Overall Training Time: {}".format(toc-tic))

--------------------------------------------------
Training step: 0 - Loss = [0.00016152348]
predition:  [[[  9.18491965e-05  -3.22693197e-04   3.86958118e-05 ...,  -2.41205594e-04
    -9.12903771e-05  -3.15538821e-04]]

 [[ -2.69229401e-05  -3.19823338e-04   2.02388515e-05 ...,  -3.14717809e-04
    -1.58045025e-04  -3.40541110e-04]]

 [[ -8.67193230e-05  -3.60489515e-04   1.61020274e-06 ...,  -3.05083508e-04
    -1.92323016e-04  -2.66470260e-04]]

 ..., 
 [[  3.75384757e-04   5.85141547e-04  -7.98567642e-04 ...,   9.38545918e-04
     9.32887299e-04   1.42529013e-03]]

 [[  7.11640266e-04  -8.82901732e-04  -6.55941040e-04 ...,   5.82676856e-04
     7.50226933e-04   9.81898588e-04]]

 [[  3.57023872e-04  -4.51177938e-04  -3.25748920e-04 ...,   4.95364963e-04
     7.56236106e-04   9.29943704e-04]]]
--------------------------------------------------
Training step: 50 - Loss = [0.00016099861]
--------------------------------------------------
Training step: 100 - Loss = [0.00016038489]
pre

Final Result for LSTM + Dropout + LSTM

In [11]:
print("Original data: ", y_train)
print("predition: ", pred)

Original data:  [[[ 0.00648234  0.00844653  0.00122324 ...,  0.00146413 -0.00024131
    0.01057648]]

 [[ 0.00655017  0.00164062  0.00814498 ..., -0.00818713  0.00772387
    0.00747556]]

 [[ 0.00532215 -0.01181034 -0.00020198 ..., -0.00619104 -0.00502994
   -0.00924658]]

 ..., 
 [[ 0.0073006  -0.00038497  0.01231446 ...,  0.00338149 -0.00349922
    0.02457467]]

 [[ 0.00362384  0.01964107  0.02107092 ...,  0.02420343  0.02243465
    0.01225619]]

 [[ 0.00478426 -0.00898927 -0.01287097 ..., -0.00358959 -0.00725052
    0.01406067]]]
predition:  [[[  5.38497022e-04  -3.37411967e-04   5.78576653e-04 ...,  -2.84398388e-05
     1.87144766e-05  -5.41934955e-05]]

 [[  8.45671394e-04  -3.78922022e-04   9.92880403e-04 ...,   4.34507207e-05
     1.14373163e-04  -4.67834116e-05]]

 [[  1.00146152e-03  -4.04677504e-04   1.23563857e-03 ...,   6.80875388e-05
     2.33132487e-04  -7.69692333e-05]]

 ..., 
 [[  9.78645248e-04   8.58420235e-04   1.52050034e-03 ...,   3.48766233e-04
     1.31245440e-0

# Single LSTM Layer Test

In [24]:
# Parameters
learning_rate = 0.0001
training_epochs = 10000
display_step = 100

# Network Parameters
hidden_dim = 10
batch_size = 1
source_sequence_length = 500
num_tickers = 10
_time_major = True 

"""
 If true, these Tensors must be shaped [max_time, batch_size, depth]. 
 If false, these Tensors must be shaped [batch_size, max_time, depth]. 
 Using time_major = True is a bit more efficient
 
 To make the shape consistent, change the hidden_dim from 4 to 10. Then we get the shape (500, ?, 10)
"""

# Reset the graph
tf.reset_default_graph()

# Graph Placeholders
encoder_inputs = tf.placeholder(shape=(source_sequence_length, None, num_tickers), 
                                dtype=tf.float64, name='encoder_inputs')
decoder_targets = tf.placeholder(shape=(source_sequence_length, None, num_tickers), 
                                 dtype=tf.float64, name='decoder_targets')

# Build Encoder cell
encoder_cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_dim,
                                       forget_bias=1.0,
                                       state_is_tuple=True,
                                       activation=tf.tanh,
                                       reuse=None)

outputs, _ = tf.nn.dynamic_rnn(encoder_cell,
                               encoder_inputs,
                               sequence_length=[source_sequence_length],
                               time_major=_time_major,
                               dtype=tf.float64)
print(decoder_targets.shape)
print(outputs.shape)

# Loss Metric (unregularized)
loss = tf.losses.mean_squared_error(labels=decoder_targets, predictions=outputs)
loss = tf.reduce_mean(loss)

# Optimizer
trainer = tf.train.AdamOptimizer(learning_rate=learning_rate)
gradients = trainer.compute_gradients(loss)
gradients_clipped = [(tf.clip_by_value(t[0],-1,1),t[1]) for t in gradients]
optimizer = trainer.apply_gradients(gradients_clipped)

(500, ?, 10)
(500, ?, 10)


In [25]:
# Training function
num_steps = 500
x_train = df.values[0:, :500].T.reshape((500, 1, 10))
y_train = df.values[0:, :500].T.reshape((500, 1, 10))

tic = time.time()        # Initialize time count
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(num_steps):
        _, train_loss, pred = sess.run([optimizer, loss, outputs], feed_dict={encoder_inputs:x_train, 
                                                               decoder_targets:y_train})

        if (step % 50 == 0):
            val_loss = sess.run([loss], feed_dict={encoder_inputs:x_train, 
                                                     decoder_targets:y_train})
            
            print("-"*50)
            print("Training step: {0} - Loss = {1}".format(step, val_loss))
        if (step % 100 == 0):
#             pred = sess.run([outputs], feed_dict={encoder_inputs:x_train})
            print("predition: ", pred)

    print("="*50)
    
toc = time.time()
print("Overall Training Time: {}".format(toc-tic))

--------------------------------------------------
Training step: 0 - Loss = [0.00016431577]
predition:  [[[  5.88245232e-05  -3.26075306e-04  -1.11841901e-03 ...,   2.78347123e-04
    -7.67959855e-04  -7.81038773e-04]]

 [[  4.59992908e-04  -5.77262734e-04  -1.22846708e-03 ...,   5.35995465e-04
    -1.32365505e-03  -8.51807622e-05]]

 [[ -1.04854913e-04  -3.09512773e-04  -2.11572458e-03 ...,   5.82904241e-04
     1.02186393e-03  -1.63457995e-03]]

 ..., 
 [[  3.61842949e-03  -2.98828628e-04  -4.47722709e-03 ...,   1.12466364e-03
    -3.65029169e-03   3.99727986e-03]]

 [[  8.64124684e-03  -2.67470949e-03  -3.61403043e-03 ...,  -3.26807857e-03
    -8.42490950e-03   6.14149939e-03]]

 [[  7.13410830e-03  -1.16735181e-03  -4.39539431e-03 ...,  -2.87129104e-04
    -7.68975293e-03   5.63001273e-03]]]
--------------------------------------------------
Training step: 50 - Loss = [0.00016228556]
--------------------------------------------------
Training step: 100 - Loss = [0.00016063209]
pre

Final Result for LSTM

In [ ]:
print("Original data: ", y_train)
print("predition: ", pred)

In [ ]:
def auto_encoder_net(encoder_input, decoder_target, keep_prob_in, keep_prob_out, keep_prob_state,
                     hidden_dim, num_tickers, source_sequence_length, l2_norm=0.01, seed=235):
    
    # Build Encoder cell
    encoder_cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_dim,
                                           forget_bias=1.0,
                                           state_is_tuple=True,
                                           activation=tf.tanh,
                                           reuse=None)
    
    # Introduce Dropout to the Encoding
    encoder_dropout = tf.contrib.rnn.DropoutWrapper(encoder_cell,
                                                    input_keep_prob=keep_prob_in,
                                                    output_keep_prob=keep_prob_out,
                                                    state_keep_prob=keep_prob_state)
    
    
    # Build Simple Decoder Cell
    decoder_cell = tf.nn.rnn_cell.LSTMCell(num_units=num_tickers,
                                           forget_bias=1.0,
                                           state_is_tuple=True,
                                           activation=tf.tanh,
                                           reuse=None)

    # Compile Deep Network
    autoencoder_layers = tf.nn.rnn_cell.MultiRNNCell([encoder_dropout, decoder_cell], 
                                                 state_is_tuple=True)


    # Run Dynamic RNN
    #   encoder_outpus: [batch_size, max_time, num_units]
    #   encoder_state: [batch_size, num_units]
    outputs, state = tf.nn.dynamic_rnn(autoencoder_layers,
                                       encoder_inputs,
                                       sequence_length=[source_sequence_length],
                                       time_major=True,
                                       dtype=tf.float64)

    #fc_w = [fc_layer_0.weight, fc_layer_1.weight, fc_layer_2.weight]

    # loss
    with tf.name_scope("loss"):
        #l2_loss = tf.reduce_sum([tf.norm(w) for w in fc_w])
        
        #label = tf.one_hot(input_y, output_size)
        #cross_entropy_loss = tf.reduce_mean(
        #    tf.nn.softmax_cross_entropy_with_logits(labels=label, logits=fc_layer_2.output()),
        #    name='cross_entropy')
        #loss = tf.add(cross_entropy_loss, l2_norm * l2_loss, name='loss')
        loss = tf.losses.mean_squared_error(labels=decoder_targets, predictions=outputs)
        loss = tf.reduce_mean(loss)

        tf.summary.scalar('auto_encode_loss', loss)

    return fc_layer_2.output(), loss

In [ ]:
def train_step(loss, lr=1e-3):
    with tf.name_scope('train_step'):
        step = tf.train.AdamOptimizer(learning_rate=lr, beta1=0.9, beta2=0.999, epsilon=1e-08).minimize(loss)
    return step

def auto_encoder_training(X_train, y_train, X_val, y_val, keep_prob=0.7,
                l2_norm=0.01, seed=235, learning_rate=1e-2, epoch=20,
                batch_size=245, verbose=True):
    print("Building my Auto-Encoder Parameters: ")
    print("l2_norm={}".format(l2_norm))
    print("seed={}".format(seed))
    print("learning_rate={}".format(learning_rate))

    # define the variables and parameter needed during training
    with tf.name_scope('inputs'):
        xs = tf.placeholder(shape=[None, 32, 32, 3], dtype=tf.float32)
        ys = tf.placeholder(shape=[None, ], dtype=tf.int64)
        ss = tf.placeholder(shape=[None, 5], dtype=tf.float32)
        kp = tf.placeholder(dtype=tf.float32)
     
    output, loss = dn2387_NN(xs, ys, kp)
    predictions = predict(ss)
    iters = int(X_train.shape[0] / batch_size)
    print('number of batches for training: {}'.format(iters))

    step = train_step(loss)
    eve = evaluate(output, ys)

    iter_total = 0
    best_acc = 0
    cur_model_name = 'dn2387_{}'.format(int(time.time()))

    with tf.Session() as sess:
        merge = tf.summary.merge_all()

        writer = tf.summary.FileWriter("log/{}".format(cur_model_name), sess.graph)
        saver = tf.train.Saver()
        sess.run(tf.global_variables_initializer())
                              
        for epc in range(epoch):
            print("epoch {} ".format(epc + 1))

            for itr in range(iters):
                iter_total += 1

                training_batch_x = X_train[itr * batch_size: (1 + itr) * batch_size]
                training_batch_y = y_train[itr * batch_size: (1 + itr) * batch_size]
                
                #import pdb
                #pdb.set_trace()
                _, cur_loss = sess.run([step, loss], feed_dict={xs: training_batch_x, ys: training_batch_y, kp: keep_prob})

                if iter_total % 10 == 0:
                    # do validation
                    valid_eve, merge_result = sess.run([eve, merge], feed_dict={xs: X_val, ys: y_val, kp:1.0})
                    valid_acc = 100 - valid_eve * 100 / y_val.shape[0]
                    if verbose:
                        print('{}/{} loss: {} validation accuracy : {}%'.format(
                            batch_size * (itr + 1),
                            X_train.shape[0],
                            cur_loss,
                            valid_acc))

                    # save the merge result summary
                    writer.add_summary(merge_result, iter_total)

                    # when achieve the best validation accuracy, we store the model paramters
                    if valid_acc > best_acc:
                        print('Best validation accuracy! iteration:{} accuracy: {}%'.format(iter_total, valid_acc))
                        best_acc = valid_acc
                        saver.save(sess, 'model/{}'.format(cur_model_name))

    print("Training ends. The best valid accuracy is {}. Model named {}.".format(best_acc, cur_model_name))
    return